In [3]:
# load .env file
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [4]:
from google import genai
from pydantic import BaseModel, Field

client = genai.Client()

class ArticleSummarySchema(BaseModel):
    title: str = Field(..., title="Title of the news article rewritten to include required words")
    summary: str = Field(..., title="Summary of the news article rewritten to include required words")

def generate_detail_summary(text: str, required_words: str) -> ArticleSummarySchema:
    response = client.models.generate_content( 
        model='gemini-2.0-flash', 
        contents=f'''Given total news article deliminated in tripple backticks. 
                        Generate a title of the topic and short summary of the following news article. 
                        The title and summary must include the following words: {required_words}.
                        You can skip to include the words in the title and summary if it makes the meaning incorrect.
                        You must write in very simple words at A1 level. The summary must be around 100 words.
                        And the summary must keep the exact meaning of the original article.
                        ``` TEXT TO SUMMARIZE HERE
                            {text}
                        ```''', 
        config={ 
            'response_mime_type': 'application/json',
            'response_schema': ArticleSummarySchema, 
            'temperature': 0.8,
        }, 
    )

    return response.parsed